In [2]:
import os
import pandas as pd
import io
import json
import itertools

#print(os.getcwd())
csvfiles = list(filter(lambda fn: fn.startswith("Alchemy") and fn.endswith(".csv"),os.listdir()))
print(csvfiles)

dfStd = pd.read_csv("Alchemy Ingredients - Standard.csv",encoding="utf-8")
dfCC = pd.read_csv("Alchemy Ingredients - Creation Club.csv",encoding="utf-8")

print(dfStd.head())
print(dfCC.head())

effects = {}
with io.open("effects.json",mode="r",encoding="utf-8") as effectsJson:
    effects = json.load(effectsJson)

maxWidth = 0
for key,values in effects.items():
    for v in values:
        print(f"{key}: {v}")
        maxWidth = max(maxWidth,len(v))
print()

categories = []
categories.append(dfStd)
categories.append(dfCC)

ingredients = pd.concat(categories,ignore_index=True,sort=False)
categories = None
print(ingredients.head())
print(ingredients.shape)

['Alchemy Ingredients - Creation Club-Backup.csv', 'Alchemy Ingredients - Quest.csv', 'Alchemy Ingredients - Standard-Backup.csv', 'Alchemy Ingredients - Creation Club.csv', 'Alchemy Ingredients - Standard.csv', 'Alchemy Ingredients - Quest-Backup.csv']
         ID                 Name          Effect1              Effect2  \
0  SXXXXXX0  Ingredient 0's Name  restore stamina       fortify health   
1  SXXXXXX1  Ingredient 1's Name   restore health  fortify conjuration   
2  SXXXXXX2  Ingredient 2's Name   restore health       fortify health   
3  SXXXXXX3  Ingredient 3's Name  restore magicka        fortify block   
4  SXXXXXX4  Ingredient 4's Name  restore magicka  fortify destruction   

                Effect3                   Effect4  \
0    fortify one-handed      damage magicka regen   
1        fortify health      damage magicka regen   
2  damage stamina regen  lingering damage magicka   
3             paralysis           fortify magicka   
4       fortify magicka             